## Notebook for the gene enrichment analysis of Cancer Joanito Epithelial Cells
**Developed by:** Anna Maguza  
**Institute of Computational Biology - Computational Health Centre - Hemlholtz Munich**  
**Date:** 4th June 2023  

#### Load required packages

In [156]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import seaborn as sns
import numpy as np

#### Setup Cells


In [157]:
%matplotlib inline

In [158]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.2.2 statsmodels==0.13.5 pynndescent==0.5.8


#### Upload Healthy and Cancer Data

In [159]:
input = '/Users/anna.maguza/Desktop/Data/Processed_datasets/Cancer_dataset_integration/Datasets integration/Epithelial_healthy_cancer/Integrated_epithelial_healthy_and_cancer.h5ad' 
adata = sc.read_h5ad(input)

In [160]:
adata.raw.X

<191909x26187 sparse matrix of type '<class 'numpy.float32'>'
	with 252680176 stored elements in Compressed Sparse Row format>

In [161]:
adata = adata.raw.to_adata()

In [162]:
input = '/Users/anna.maguza/Desktop/Data/Processed_datasets/Cancer_dataset_integration/Datasets integration/Joanito_epithelial/Integrated_Joanito_epithelial_datasets_5000_2.h5ad'
adata_cancer = sc.read_h5ad(input)
adata_cancer.raw.X

<35714x33287 sparse matrix of type '<class 'numpy.float32'>'
	with 66145963 stored elements in Compressed Sparse Row format>

In [163]:
adata_cancer = adata_cancer.raw.to_adata()

In [183]:
input = '/Users/anna.maguza/Desktop/Data/Processed_datasets/Cancer_dataset_integration/Datasets integration/healthy_epithelial/Integrated_Healthy_epithelial_datasets_5000.h5ad'
adata_healthy = sc.read_h5ad(input)
adata_healthy.raw.X

<156195x26442 sparse matrix of type '<class 'numpy.float32'>'
	with 202376650 stored elements in Compressed Sparse Row format>

In [184]:
adata_healthy = adata_healthy.raw.to_adata()

### Prepare Gene Sets

In [164]:
Krebs_cycle_genes = ['ACO2', 'CS', 'FH', 'MDH1', 'OGDH', 'PDHA1', 'PDHA2', 'SDHC', 'SUCLG1', 'ACLY', 'ACO1', 'DLAT', 'DLD',
                     'DLST', 'IDH1', 'IDH2', 'IDH3A', 'IDH3B', 'IDH3G', 'MDH2', 'MPC1', 'OGDHL', 'PC', 'PCK1', 'PCK2',
                     'PDHB', 'SDHA', 'SDHB', 'SDHD', 'SUCLA2', 'SUCLG2']

In [165]:
mitochondrial_genes = ['MRPL1', "MRPL2", "MRPL3", "MRPL4", "MRPL9", "MRPL10", "MRPL11", "MRPL12", "MRPL13", "MRPL14", "MRPL15",
                       "MRPL16", "MRPL17", "MRPL18", "MRPL19", "MRPL20", "MRPL21", "MRPL22", "MRPL23", "MRPL24", "MRPL27", "MRPL28", 
                       "MRPL30", "MRPL32", "MRPL33", "MRPL34", "MRPL35", "MRPL36", "MRPL37", "MRPL38", "MRPL39", "MRPL40","MRPL41",
                       "MRPL42", "MRPL43", "MRPL44", "MRPL45", "MRPL46", "MRPL47", "MRPL48", "MRPL49", "MRPL50", "MRPL51", "MRPL52",
                       "MRPL53", "MRPL54", "MRPL55", "MRPL57",
                       "MRPS2", "MRPS5", "MRPS6", "MRPS7", "MRPS9", "MRPS10", "MRPS11", "MRPS12", "MRPS14", "MRPS15", "MRPS16", "MRPS17",
                       "MRPS18A", "MRPS18B", "MRPS18C", "MRPS21", "MRPS22", "MRPS23", "MRPS24", "MRPS25", "MRPS26", "MRPS27", "MRPS28", "MRPS30", 
                       "MRPS31", "MRPS33", "MRPS34", "MRPS35", "MRPS36", 'DAP3',
                       "MT-ND1", "MT-ND2", "MT-COX1", "MT-COX2" ,"MT-ATP8", "MT-ATP6", "MT-COX3", "MT-ND3", "MT-ND4L", "MT-ND4", "MT-ND5", "MT-ND6", "MT-CYTB",
                       'POLG', "POLG2", "SOD2", "MFN1", "MFN2", "OPA1", "PINK1", "PARK7", "PARK2", "SDHA", "SDHB", "SDHC", "SDHD", "NDUFS1", "NDUFS2", "NDUFS3", "NDUFS4", "NDUFS7", "NDUFS8", "NDUFA1", "NDUFA2", "NDUFA9"]

In [166]:
glycolysis_genes = ['ALDOA', 'BPGM', 'ENO1', 'ENO2', 'GAPDH', 'GPI', 'HK1', 'HK2', 'HKDC1', 'PFKL', 'PFKM', 'PGAM1', 
                    'PGAM2', 'PGAM4', 'PGK1', 'PKLR', 'PKM', 'TPI1']

In [167]:
gluconeogenesis = ['ALDOA', 'ALDOB', 'ALDOC', 'ENO1', 'ENO2', 'ENO3', 'FBP1', 'FBP2', 'GAPDH', 'GAPDHS', 
                   'GOT1', 'GOT2', 'GPI', 'MDH1', 'MDH2', 'PC', 'PCK1', 'PCK2', 'PFKFB1', 'PGAM1', 'PGAM2', 
                   'PGK1', 'PRKACA', 'PRKACB', 'PRKACG', 'SLC25A1', 'SLC25A10', 'SLC25A11', 'SLC25A12', 'SLC25A13', 'TPI1']

In [168]:
lipogenesis = ['ACACA', 'ACBD3', 'AGPAT2', 'ANGPTL4', 'APOB', 'AR', 'BRCA1', 'CD68', 'CHUK', 'EIF2AK3', 'ENHO',
               'EPAS1', 'FASN', 'FTO', 'GHRL', 'HK2', 'HNF4A', 'HOXB13', 'IDH1', 'IDH2', 'IGF1', 'INSIG1', 'LPIN1',
               'MIR1-1', 'MIR185', 'MIR206', 'MIR29A', 'MIR342', 'MIR613', 'MLXIPL', 'MORC2', 'MRC1', 'MTOR', 'NAMPT',
               'NEWENTRY', 'NOS3', 'NR0B1', 'NR1H2', 'NR1H3', 'OLR1', 'PFKFB2', 'PIK3CA', 'PPARD', 'PPARG', 'PPARGC1A',
               'PRKACA', 'PRL', 'RPS6', 'SCD', 'SHBG', 'SMARCD3', 'SREBF1', 'TRA2B', 'TRIB1', 'TUT1']

In [169]:
line1 = ['AICDA', 'APC', 'APOBEC3C', 'ATM', 'CDKN2A', 'EDA', 'ERCC1', 'ERCC4', 'ETS1', 'EVC', 'EVC2', 'GSTM1',
         'H19', 'HLA-G', 'IGF2', 'IGF2BP3', 'IGFBP7', 'L1RE1', 'L1TD1', 'LINC01587', 'MAL', 'MECP2', 'MET', 'MGMT',
         'MLH1', 'MTHFR', 'MTRR', 'NEWENTRY', 'PADI4', 'PEG3', 'PIWIL2', 'PROM1', 'SAMHD1', 'SLIT2', 'STK32B', 'TERT',
         'TINF2',
         'ORF1', 'ORF2', 'APOBEC3', 'TREX1', 'MOV10', 'PIWIL1', "PIWIL2", "PIWIL3", "PIWIL4", "DICER1", "AGO2", "EIF2C2"]



In [170]:
IL10 = ['IL10', 'IL10RA', 'IL10RB']
IL23 = ['IL23A', 'IL12B', 'IL12RB1', 'IL23R']
TNF_alpha = ['TNF', 'TNFRSF1A', 'TNFRSF1B']
IFN_gamma = ['IFNG', 'IFNGR-1', 'IFNGR-2']
IFN_alpha = ['IFNA1', 'IFNAR1', 'IFNAR2']

In [171]:
# Proliferation genes
PI3K_AKT = ['AKT1', 'AKT2', 'AKT3', 'BAD', 'CASP9', 'CDKN1A', 'CDKN1B', 'CHUK', 'CKB', 'ESR1', 'FOXO1', 
            'FOXO3', 'FOXO4', 'GSK3A', 'GSK3B', 'HIF1A', 'HSP90AA1', 'INPP5D', 'INPPL1', 'INSR', 'KPNA1', 
            'MAP3K5', 'MAPKAP1', 'MLST8', 'MTOR', 'PDPK1', 'PGR', 'PHLPP1', 'PHLPP2', 'PIK3CA', 'PIK3CB', 
            'PIK3CD', 'PIK3CG', 'PIK3R1', 'PIK3R2', 'PIK3R3', 'PPP2CA', 'PPP2CB', 'PPP2R1A', 'PPP2R1B',
              'PPP2R2A', 'PRKACA', 'PRKDC', 'PTEN', 'RAF1', 'RICTOR', 'SFN', 'SLC2A4', 'SP1', 'SRC', 
              'TBC1D4', 'VEGFA', 'YWHAB', 'YWHAE', 'YWHAH', 'YWHAQ', 'YWHAZ']

MAPK_Erk = ["ARAF", "BRAF", "DUSP6", "GPER1", "HRAS", "KRAS", "KSR1", "KSR2", "MAP2K1", "MAP2K2", 
            "MAP3K1", "MAPK1", "MAPK3", "MKNK1", "MKNK2", "NRAS", "PEBP1", "PRKCD", "PTK2B", "RAF1",
              "RPS6KA1", "RPS6KA2", "RPS6KA3", "RPS6KA4", "RPS6KA5", "RPS6KA6", "S1PR1", "SHC1", "SMAD1", 
              "SPP1", "TSC2"]

MTOR = ["AKT1", "AKT1S1", "AKT2", "AKT3", "ATG13", "BNIP3", "BRAF", "CAB39", "CAB39L", "CASTOR1", "CASTOR2",
        "CCNE1", "CDK2", "CLIP1", "CXCL8", "CYCS", "DDIT4", "DEPDC5", "DEPTOR", "EEF2", "EEF2K", "EIF4A1",
        "EIF4B", "EIF4E", "EIF4E1B", "EIF4E2", "EIF4EBP1", "FBXW11", "FKBP1A", "FLCN", "FNIP1", "FNIP2",
        "HIF1A", "HRAS", "IGF1", "IKBKB", "INS", "IRS1", "LAMTOR1", "LAMTOR2", "LAMTOR3", "LAMTOR4", 
        "LAMTOR5", "LPIN1", "MAP2K1", "MAP2K2", "MAPK1", "MAPK3", "MAPKAP1", "MIOS", "MLST8", "MTOR",
        "NPRL2", "NPRL3", "NRAS", "PDCD4", "PDPK1", "PGF", "PIK3CA", "PIK3CB", "PIK3CD", "PIK3CG",
        "PIK3R1", "PIK3R2", "PIK3R3", "PIK3R5", "PLD1", "PLD2", "PML", "POLDIP3", "PPARGC1A", "PRKAA1",
        "PRKAA2", "PRKCA", "PRR5", "PRR5L", "PXN", "RAC1", "RAF1", "RB1CC1", "RHEB", "RHOA", "RICTOR",
        "RPS6", "RPS6KA1", "RPS6KA2", "RPS6KA3", "RPS6KA6", "RPS6KB1", "RPS6KB2",
        "RRAGA", "RRAGB", "RRAGC", "RRAGD","RPTOR", "RRN3", "SESN2", "SFN", "SGK1", 
        "SLC7A5", "SREBF1", "SSPOP", "STK11", "STRADA", "TBC1D7", "TELO2", "TSC1", "TSC2", "TTI1", 
        "ULK1", "ULK2", "ULK3", "VEGFA", "VEGFB", "VEGFC", "VEGFD", "WDR24", "WDR59", "YWHAB", "YWHAE", 
        "YWHAG", "YWHAH", "YWHAQ", "YWHAZ", "YY1"]

In [173]:
SINE = ['ADAR', 'APOBEC3', 'L1 ORF2p', 'SAMHD1', 'TREX1', 'DICER1', 'AGO2', 'EIF2C2']
Ty1 = ['TYB1', 'TYB2', 'SPT3', 'RTT101', 'RTT107', "MMS1", "RAD27", "RAD52", 'XRN1', 'DDC1', 'MEC3', 'RAD17']
Ty3 = ['TYC1', 'TYC2', 'RPC53', 'RPC37', 'RTT101', 'MMS1', 'RTT107', 'FZF1', 'GAL11']
HERVs = ["APOBEC3G", "APOBEC3F", "APOBEC3B", "APOBEC3D", "APOBEC3H", "TRIM28", "PIWIL1", "PIWIL2", "PIWIL4", "DNMT1", "DNMT3A", "DNMT3B"]

### Calculate gene enrichment score and create UMAPs

In [175]:
sc.tl.score_genes(adata_cancer, Krebs_cycle_genes, score_name = "Krebs_cycle_genes")
sc.tl.score_genes(adata_cancer, mitochondrial_genes, score_name = "Mitochondrial_genes")
sc.tl.score_genes(adata_cancer, glycolysis_genes, score_name = "Glycolysis_genes")
sc.tl.score_genes(adata_cancer, gluconeogenesis, score_name = "Gluconeogenesis_genes")
sc.tl.score_genes(adata_cancer, lipogenesis, score_name = "Lipogenesis_genes")
sc.tl.score_genes(adata_cancer, line1, score_name = "LINE1_genes")
sc.tl.score_genes(adata_cancer, IL10, score_name = "IL-10")
sc.tl.score_genes(adata_cancer, IL23, score_name = "IL-23")
sc.tl.score_genes(adata_cancer, TNF_alpha, score_name = "TNF_alpha")
sc.tl.score_genes(adata_cancer, IFN_gamma, score_name = "IFN_gamma")
sc.tl.score_genes(adata_cancer, IFN_alpha, score_name = "IFN_alpha")
sc.tl.score_genes(adata_cancer, PI3K_AKT, score_name = "PI3K_AKT")
sc.tl.score_genes(adata_cancer, MAPK_Erk, score_name = "MAPK_Erk")
sc.tl.score_genes(adata_cancer, MTOR, score_name = "MTOR")
sc.tl.score_genes(adata_cancer, SINE, score_name = "SINE")
sc.tl.score_genes(adata_cancer, Ty1, score_name = "Ty1")
#sc.tl.score_genes(adata_cancer, Ty3, score_name = "Ty3")
sc.tl.score_genes(adata_cancer, HERVs, score_name = "HERVs")    

computing score 'Krebs_cycle_genes'
    finished: added
    'Krebs_cycle_genes', score of gene set (adata.obs).
    499 total control genes are used. (0:00:00)
computing score 'Mitochondrial_genes'
    finished: added
    'Mitochondrial_genes', score of gene set (adata.obs).
    446 total control genes are used. (0:00:00)
computing score 'Glycolysis_genes'
    finished: added
    'Glycolysis_genes', score of gene set (adata.obs).
    498 total control genes are used. (0:00:00)
computing score 'Gluconeogenesis_genes'
    finished: added
    'Gluconeogenesis_genes', score of gene set (adata.obs).
    650 total control genes are used. (0:00:00)
computing score 'Lipogenesis_genes'
    finished: added
    'Lipogenesis_genes', score of gene set (adata.obs).
    750 total control genes are used. (0:00:00)
computing score 'LINE1_genes'
    finished: added
    'LINE1_genes', score of gene set (adata.obs).
    798 total control genes are used. (0:00:00)
computing score 'IL-10'
    finished: adde

In [ ]:
sc.set_figure_params(figsize = (8, 8), dpi=300)
sc.pl.umap(adata_cancer, color= ['Predicted_Label', 'LINE1_genes', "SINE", "Ty1", "HERVs"], color_map = "Spectral_r", size = 12, frameon = False, ncols= 3)

In [ ]:
sc.set_figure_params(figsize = (8, 8), dpi=300)
sc.pl.umap(adata_cancer, color= ['Predicted_Label', 'Glycolysis_genes', 'Gluconeogenesis_genes', 'Lipogenesis_genes', 'LINE1_genes', 'IL-23', 'IFN_gamma'], color_map = "Spectral_r", size = 12, frameon = False, ncols= 3)

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata_cancer, color= ['Predicted_Label', 'PI3K_AKT', 'MAPK_Erk', 'MTOR'], color_map = "Spectral_r", size = 4, frameon = False, ncols= 3)

### Healthy and cancer data

In [179]:
sc.tl.score_genes(adata, Krebs_cycle_genes, score_name = "Krebs_cycle_genes")
sc.tl.score_genes(adata, mitochondrial_genes, score_name = "Mitochondrial_genes")
sc.tl.score_genes(adata, glycolysis_genes, score_name = "Glycolysis_genes")
sc.tl.score_genes(adata, gluconeogenesis, score_name = "Gluconeogenesis_genes")
sc.tl.score_genes(adata, lipogenesis, score_name = "Lipogenesis_genes")
sc.tl.score_genes(adata, line1, score_name = "LINE1_genes")
sc.tl.score_genes(adata, IL10, score_name = "IL-10")
sc.tl.score_genes(adata, IL23, score_name = "IL-23")
sc.tl.score_genes(adata, TNF_alpha, score_name = "TNF_alpha")
sc.tl.score_genes(adata, IFN_gamma, score_name = "IFN_gamma")
sc.tl.score_genes(adata, IFN_alpha, score_name = "IFN_alpha")
sc.tl.score_genes(adata, PI3K_AKT, score_name = "PI3K_AKT")
sc.tl.score_genes(adata, MAPK_Erk, score_name = "MAPK_Erk")
sc.tl.score_genes(adata, MTOR, score_name = "MTOR")
sc.tl.score_genes(adata, SINE, score_name = "SINE")
sc.tl.score_genes(adata, Ty1, score_name = "Ty1")
#sc.tl.score_genes(adata, Ty3, score_name = "Ty3")
sc.tl.score_genes(adata, HERVs, score_name = "HERVs")

computing score 'Krebs_cycle_genes'
    finished: added
    'Krebs_cycle_genes', score of gene set (adata.obs).
    349 total control genes are used. (0:00:04)
computing score 'Mitochondrial_genes'
    finished: added
    'Mitochondrial_genes', score of gene set (adata.obs).
    495 total control genes are used. (0:00:03)
computing score 'Glycolysis_genes'
    finished: added
    'Glycolysis_genes', score of gene set (adata.obs).
    500 total control genes are used. (0:00:03)
computing score 'Gluconeogenesis_genes'
    finished: added
    'Gluconeogenesis_genes', score of gene set (adata.obs).
    799 total control genes are used. (0:00:03)
computing score 'Lipogenesis_genes'
    finished: added
    'Lipogenesis_genes', score of gene set (adata.obs).
    749 total control genes are used. (0:00:03)
computing score 'LINE1_genes'
    finished: added
    'LINE1_genes', score of gene set (adata.obs).
    800 total control genes are used. (0:00:03)
computing score 'IL-10'
    finished: adde

In [180]:
sc.pp.neighbors(adata, use_rep='X_scANVI')
sc.tl.umap(adata, min_dist = 0.4, spread = 4, random_state = 1712)

computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:18)
computing UMAP


/Users/anna.maguza/miniconda3/envs/scanpy_env/lib/python3.9/site-packages/umap/umap_.py:1356: RuntimeWarning: divide by zero encountered in power
  return 1.0 / (1.0 + a * x ** (2 * b))


    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:02:34)


In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, color= ['Unified_Cell_States', 'Study_name', 'Diagnosis', 'Mitochondrial_genes', 'Glycolysis_genes', 'Gluconeogenesis_genes', 'LINE1_genes', 'IL-10', 'IL-23', 'TNF_alpha', 'IFN_gamma', 'IFN_alpha'], color_map = "Spectral_r", size = 1, frameon = False, ncols= 3)

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, color= ['Unified_Cell_States', 'Study_name', 'LINE1_genes', "SINE", 'Ty1', 'HERVs'], color_map = "Spectral_r", size = 1, frameon = False, ncols= 3)

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.umap(adata, color= ['Unified_Cell_States', 'Study_name', 'Diagnosis', 'PI3K_AKT', 'MAPK_Erk', 'MTOR'], color_map = "Spectral_r", size = 1, frameon = False, ncols= 3)

### Create barplots with sex

In [ ]:
# create a barplot with sex (how many male patients and how many female patients)
import matplotlib.pyplot as plt

# Assuming your Anndata object is named `adata_cancer` and the sex information is in the column 'Sex'
patient_sex_counts = adata.obs.groupby('Sample_ID')['Sex'].first().value_counts()

plt.bar(patient_sex_counts.index, patient_sex_counts.values)
plt.xlabel('Sex')
plt.ylabel('Number of Samples')
plt.title('Patient Sex Distribution')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming your Anndata object is named `adata_cancer` and the sex information is in the column 'Sex'
sex_counts = adata.obs['Sex'].value_counts()

plt.bar(sex_counts.index, sex_counts.values)
plt.xlabel('Sex')
plt.ylabel('Count')
plt.title('Patient Sex Distribution')
plt.show()

### Create heatmaps

In [129]:
line1 = ["AGO2", 'AICDA', 'ATM', 'CDKN2A', "DICER1", 'EDA', 'ETS1', 'EVC', 'EVC2', 'GSTM1',
         'HLA-G', 'IGF2', 'IGFBP7', 'IGF2BP3', 'L1TD1', 'MAL', 'MECP2', 'MET', 'MGMT',
         'MLH1', 'MOV10', 'PADI4', 'PEG3', 'PIWIL1', 'PIWIL2', "PIWIL3", "PIWIL4", 'PROM1', 'SAMHD1', 'SLIT2', 'STK32B', 'TERT',
         'TINF2', 'TREX1']

In [143]:
# Delete microfold cells and enterochromaffin cells
adata_cancer = adata_cancer[~adata_cancer.obs['Predicted_Label'].isin(['Microfold cell', 'Enterochromaffin cells', 'L cells', 'Epithelial cells'])]

In [ ]:
adata_cancer.obs['Predicted_Label'].value_counts()

In [ ]:
# create a heatmap of the top 10 genes
sc.set_figure_params(dpi=300)
sc.pl.heatmap(adata_cancer, var_names = line1, groupby = 'Predicted_Label', 
              log = True, use_raw = False, cmap = 'Spectral_r', swap_axes=True, 
              figsize=(60,25))

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.matrixplot(adata_cancer, line1, 'Predicted_Label', dendrogram=False, cmap='Blues', standard_scale='var', colorbar_title='column scaled\nexpression')

In [172]:
DESMOSOME = ["DSG1", "DSG2", "DSG3", "DSG4", "DSC1", "DSC2", "DSC3", "JUP", "PKP1", "PKP2", "PKP3", "PKP4", "DSP", "EVPL", "PERP"]
CDH = ["CDH1", "CDH2", "CTNNA1", "CTNNA2", "CTNNA3", "CTNNB1", "CTNND1", "JUP", "LEF1", "TCF7", "TCF7L1", "TCF7L2", "GSK3B", "APC", "AXIN1", "AXIN2"]

In [188]:
# Rename Unified Cell States column to Cell Type
adata_healthy.obs.rename(columns = {'Unified Cell States': 'Unified_Cell_States'}, inplace = True)

In [ ]:
# create a heatmap 
sc.set_figure_params(dpi=300)
sc.pl.heatmap(adata_healthy, var_names = DESMOSOME, groupby = 'Unified_Cell_States', 
              log = True, use_raw = False, cmap = 'Spectral_r', swap_axes=True, 
              figsize=(60,25))

In [ ]:
# create a heatmap 
sc.set_figure_params(dpi=300)
sc.pl.heatmap(adata_healthy, var_names = CDH, groupby = 'Unified_Cell_States', 
              log = True, use_raw = False, cmap = 'Spectral_r', swap_axes=True, 
              figsize=(60,25))

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.matrixplot(adata_healthy, DESMOSOME, 'Unified_Cell_States', dendrogram=False, cmap='Blues', standard_scale='var', colorbar_title='column scaled\nexpression')

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.matrixplot(adata_healthy, CDH, 'Unified_Cell_States', dendrogram=False, cmap='Blues', standard_scale='var', colorbar_title='column scaled\nexpression')